# <h1><center>Machine Learning</center></h1>
### **Datos para entrenamiento, validación y prueba**


Debemos separar los conjuntos de datos en entrenamiento y pruebas, con el fin que el modelo no aprenda de memoria

 <center>
 <img src="https://drive.google.com/uc?export=view&id=1gfqjFvxzL9XSKVFlxViKsiI7n-T1IJFL" style="display:block; margin:auto" >
 </center>

 Sin embargo también es necesario crear otro subconjunto de datos, llamado datos de validación

 <center>
 <img src="https://drive.google.com/uc?export=view&id=1RnfSvMdrjyHoloL8iw3KxDcY6Zs8iViw" style="display:block; margin:auto" >
 </center>

Algunos porcentajes de distribución entre los datos de entrenamiento, validación y prueba

<center>
 <img src="https://drive.google.com/uc?export=view&id=1B8mRalZntKV0hWIPqTTwPGpbwmnl901k" style="display:block; margin:auto" >
 </center>

Los datos se deben seleccionar de manera aleatoria

<center><h1>Dataset a utilizar</h1></center>

Usaremos el siguiente dataset encontrado en Kaggle, en el siguiente link: </br>
https://www.kaggle.com/datasets/mfarhaannazirkhan/heart-dataset

Se ha seleccionado este dataset, por estar más actualizado e incluir referencias

Acorde con la descripción del dataset tendremos las siguientes columnas:


1. age: Age of the patient (Numeric).
2. sex: Gender of the patient. Values: 1 = male, 0 = female.
3.     cp: Chest pain type. Values: 0 = Typical angina, 1 = Atypical angina, 2 = Non-anginal pain, 3 = Asymptomatic.
4. trestbps: Resting Blood Pressure (in mm Hg) (Numeric).
5. chol: Serum Cholesterol level (in mg/dl) (Numeric).
6. fbs: Fasting blood sugar > 120 mg/dl. Values: 1 = true, 0 = false.
7. restecg: Resting electrocardiographic results. Values: 0 = Normal, 1 = ST-T wave abnormality, 2 = Left ventricular hypertrophy.
8. thalach: Maximum heart rate achieved (Numeric).
9. exang: Exercise-induced angina. Values: 1 = yes, 0 = no.
10. oldpeak: ST depression induced by exercise relative to rest (Numeric).
11. slope: Slope of the peak exercise ST segment. Values: 0 = Upsloping, 1 = Flat, 2 = Downsloping.
12. ca: Number of major vessels (0-3) colored by fluoroscopy. Values: 0, 1, 2, 3.
13. thal: Thalassemia types. Values: 1 = Normal, 2 = Fixed defect, 3 = Reversible defect.
14. target: Outcome variable (heart attack risk). Values: 1 = more chance of heart attack, 0 = less chance of heart attack.






In [ ]:
#cargamos el dataset
import pandas as pd

df =pd.read_csv('cleaned_merged_heart_dataset.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalachh,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
#observamos el tamaño del dataset
df.shape


(1888, 14)

In [ ]:
#partimos en entrenamiento y test
from sklearn.model_selection import train_test_split

#definir variables predictoras
X = df.drop("target", axis=1)
y = df["target"]

#dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("filas entrenamiento", X_train.shape[0])
print("filas prueba", X_test.shape[0])

filas entrenamiento 1510
filas prueba 378


In [ ]:
#observamos los tamaños de entrenamiento y prueba

In [ ]:
#vamos a hacer validación cruzada con el primer modelo
#tomemos LogisticRegression


#importemos las librerías
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

#definamos las metricas a medir, las cuales serán
metricas = ['accuracy', 'precision', 'recall', 'f1']

#sacamos las columnas numericas
categorical_cols = ['cp', 'restecg', 'slope', 'ca', 'thal']
numeric_cols = [c for c in X_train.columns if c not in categorical_cols]

#preprocesamiento, escalar sólo las numericas; categoricas pasan sin transformar
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols)
    ],
    remainder='passthrough'
)

#vamos a crear un pipeline con el fin de realizar escalado por cada fold
pipeline = Pipeline(
    steps=[
        ('prep', preprocessor),
        ('clf', LogisticRegression())
    ]
)

#creamos los folds
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

#realizamos la validación cruzada
scores_logreg = cross_validate(
    pipeline, X_train, y_train, scoring = metricas, cv=cv
)

#imprimimos los valores obtenidos
print("Resultados de las metricas para regresion logistica")
for m in metricas:
  vals = scores_logreg[f"test_{m}"]
  print(f"{m:>9}: {vals.mean():.3f}")

Resultados de las metricas para regresion logistica
 accuracy: 0.759
precision: 0.746
   recall: 0.808
       f1: 0.776


In [ ]:

# vamos a hacer validación cruzada con el segundo modelo
# tomemos RandomForestClassifier

# importemos las librerías
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

# pipeline: preprocesamiento (igual que antes) + Random Forest
rf_pipeline = Pipeline([
    ("prep", preprocessor),  # StandardScaler en numéricas; resto pasa igual
    ("clf", RandomForestClassifier(random_state=42, n_jobs=-1))
])

# validación cruzada sobre el CONJUNTO DE ENTRENAMIENTO
scores_rf = cross_validate(
    rf_pipeline, X_train, y_train,
    scoring=metricas, cv=cv, n_jobs=-1, return_train_score=False
)

# impresión de resultados
print("Resultados CV (5-fold) - Random Forest (sin One-Hot)")
for m in metricas:
    vals = scores_rf[f"test_{m}"]
    print(f"{m:>9}: {vals.mean():.3f} ± {vals.std():.3f}")


Resultados CV (5-fold) - Random Forest (sin One-Hot)
 accuracy: 0.966 ± 0.008
precision: 0.970 ± 0.009
   recall: 0.963 ± 0.012
       f1: 0.967 ± 0.008


In [ ]:
# vamos a hacer validación cruzada con el tercer modelo
# tomemos Support Vector Machines

from sklearn.svm import SVC
from sklearn.model_selection import cross_validate

# pipeline: preprocesamiento + SVM
svm_pipeline = Pipeline([
    ("prep", preprocessor),   # Escala numéricas; categóricas pasan igual (sin OHE aún)
    ("clf", SVC())            # kernel='rbf', C=1.0, gamma='scale' por defecto
])

# validación cruzada sobre el CONJUNTO DE ENTRENAMIENTO
scores_svm = cross_validate(
    svm_pipeline, X_train, y_train,
    scoring=metricas, cv=cv, n_jobs=-1, return_train_score=False
)

# impresión de resultados
print("Resultados CV (5-fold) - SVM (sin One-Hot)")
for m in metricas:
    vals = scores_svm[f"test_{m}"]
    print(f"{m:>9}: {vals.mean():.3f} ± {vals.std():.3f}")


Resultados CV (5-fold) - SVM (sin One-Hot)
 accuracy: 0.862 ± 0.022
precision: 0.862 ± 0.011
   recall: 0.873 ± 0.043
       f1: 0.867 ± 0.024


In [ ]:
#vamos a instalar una librería necesaria para poder hacer cross validate con keras
!pip install -U scikit-learn scikeras tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 73.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.


In [ ]:
# vamos a hacer validación cruzada con el cuarto modelo
# tomemos Redes neuronales de keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_validate
import numpy as np
import tensorflow as tf

# Opcional: semillas para reproducibilidad (puedes omitir si no te interesa)
np.random.seed(42)
tf.random.set_seed(42)

# build_model: recibe 'meta' con info del pipeline (como n_features_in_)
def build_model(meta):
    n_features = meta["n_features_in_"]  # número de columnas después del preprocesamiento
    model = Sequential()
    model.add(Dense(16, activation="relu", input_shape=(n_features,)))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Wrapper de Keras para usarlo como estimador sklearn
nn_clf = KerasClassifier(
    model=build_model,
    epochs=50,
    batch_size=32,
    verbose=0  # silenciar salida de Keras en CV
)

# pipeline: preprocesamiento + red neuronal
nn_pipeline = Pipeline([
    ("prep", preprocessor),  # StandardScaler en numéricas; categóricas pasan igual (sin OHE aún)
    ("clf", nn_clf)
])

# validación cruzada sobre el CONJUNTO DE ENTRENAMIENTO
scores_nn = cross_validate(
    nn_pipeline, X_train, y_train,
    scoring=metricas, cv=cv, n_jobs=1, return_train_score=False  # n_jobs=1 para evitar problemas con TF
)

# impresión de resultados
print("Resultados CV (5-fold) - Keras NN (sin One-Hot)")
for m in metricas:
    vals = scores_nn[f"test_{m}"]
    print(f"{m:>9}: {vals.mean():.3f} ± {vals.std():.3f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo

Resultados CV (5-fold) - Keras NN (sin One-Hot)
 accuracy: 0.814 ± 0.019
precision: 0.787 ± 0.014
   recall: 0.878 ± 0.029
       f1: 0.830 ± 0.019


In [ ]:
# ahora vamos a hacer one hot encoder
# en este colab, vamos a tomar un solo modelo para comparar, digamos Redes Neuronales

from sklearn.preprocessing import OneHotEncoder
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# columnas categóricas y numéricas
categorical_cols = ["cp", "restecg", "slope", "ca", "thal"]
numeric_cols = [c for c in X_train.columns if c not in categorical_cols]

# preprocesador con OHE en categóricas y escalado en numéricas
preprocessor_ohe = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(), numeric_cols)
    ],
    remainder="drop"
)

# función para construir la red
def build_model(meta):
    n_features = meta["n_features_in_"]
    model = Sequential()
    model.add(Dense(16, activation="relu", input_shape=(n_features,)))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# wrapper de Keras para usarlo en cross_validate
nn_clf_ohe = KerasClassifier(
    model=build_model,
    epochs=50,
    batch_size=32,
    verbose=0
)

# pipeline con OHE + red neuronal
nn_pipeline_ohe = Pipeline([
    ("prep", preprocessor_ohe),
    ("clf", nn_clf_ohe)
])

# validación cruzada sobre el CONJUNTO DE ENTRENAMIENTO
scores_nn_ohe = cross_validate(
    nn_pipeline_ohe, X_train, y_train,
    scoring=metricas, cv=cv, n_jobs=1, return_train_score=False
)

# impresión de resultados
print("Resultados CV (5-fold) - Keras NN CON One-Hot")
for m in metricas:
    vals = scores_nn_ohe[f"test_{m}"]
    print(f"{m:>9}: {vals.mean():.3f} ± {vals.std():.3f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo

Resultados CV (5-fold) - Keras NN CON One-Hot
 accuracy: 0.875 ± 0.026
precision: 0.867 ± 0.022
   recall: 0.896 ± 0.029
       f1: 0.882 ± 0.025
